In [3]:
import numpy as np
import scipy.stats as stats
import unittest


In [4]:


class BlockingSystemSimulation:
    def __init__(self, m, mean_service_time, mean_interarrival_time, num_customers):
        self.m = m  # number of service units
        self.mean_service_time = mean_service_time
        self.mean_interarrival_time = mean_interarrival_time
        self.num_customers = num_customers
        self.blocked_customers = 0
        self.service_times = []
        self.arrival_times = []
        self.service_start_times = []
        self.service_end_times = []

    def simulate(self):
        arrival_time = 0
        for _ in range(self.num_customers):
            # Generate interarrival time and update arrival time
            interarrival_time = np.random.exponential(self.mean_interarrival_time)
            arrival_time += interarrival_time
            self.arrival_times.append(arrival_time)

            # Remove completed services
            self.service_end_times = [end_time for end_time in self.service_end_times if end_time > arrival_time]
            

            # Check for available service units
            if len(self.service_end_times) < self.m:
                # Generate service time
                service_time = np.random.exponential(self.mean_service_time)
                self.service_times.append(service_time)
                # Service can start immediately
                service_start_time = arrival_time
                service_end_time = service_start_time + service_time
                self.service_start_times.append(service_start_time)
                self.service_end_times.append(service_end_time)
            else:
                # All service units are busy, customer is blocked
                self.blocked_customers += 1

            
           
    def fraction_blocked(self):
        return self.blocked_customers / self.num_customers

    def confidence_interval(self, confidence=0.95):
        p = self.fraction_blocked()
        n = self.num_customers
        z = stats.norm.ppf((1 + confidence) / 2)
        se = np.sqrt(p * (1 - p) / n)
        print(se)
        return p - z * se, p + z * se

if __name__ == "__main__":
    m = 10
    mean_service_time = 8
    mean_interarrival_time = 1
    num_customers = 10000
    block_list =[]
    for i in range(10+1):
        simulation = BlockingSystemSimulation(m, mean_service_time, mean_interarrival_time, num_customers)
        simulation.simulate()
        block_list.append(simulation.fraction_blocked())
        print(f"Simulation {i+1}: Blocked customers = {simulation.fraction_blocked():.4f}")

    blocked_fraction = simulation.fraction_blocked()
    ci_lower, ci_upper = simulation.confidence_interval()

    print(f"Fraction of blocked customers: {blocked_fraction:.4f}")
    print(f"95% Confidence interval for blocked fraction: ({ci_lower:.4f}, {ci_upper:.4f})")

Simulation 1: Blocked customers = 0.1153
Simulation 2: Blocked customers = 0.1250
Simulation 3: Blocked customers = 0.1171
Simulation 4: Blocked customers = 0.1265
Simulation 5: Blocked customers = 0.1118
Simulation 6: Blocked customers = 0.1188
Simulation 7: Blocked customers = 0.1117
Simulation 8: Blocked customers = 0.1261
Simulation 9: Blocked customers = 0.1192
Simulation 10: Blocked customers = 0.1162
Simulation 11: Blocked customers = 0.1194
0.003242586005027469
Fraction of blocked customers: 0.1194
95% Confidence interval for blocked fraction: (0.1130, 0.1258)


In [5]:
import numpy as np
import scipy.stats as stats

def confidence_interval_t(data, confidence=0.95):
    n = len(data)
    mean = np.mean(data)
    std_err = stats.sem(data)  # Standard error of the mean
    t_crit = stats.t.ppf((1 + confidence) / 2, df=n-1)
    h = t_crit * std_err
    return mean - h, mean + h

# Example usage:
block_list = block_list # Replace with your simulation results
ci_lower, ci_upper = confidence_interval_t(block_list)
print(f"95% t-confidence interval for mean blocked fraction: ({ci_lower:.4f}, {ci_upper:.4f})")

95% t-confidence interval for mean blocked fraction: (0.1153, 0.1223)


In [6]:
def erlang_b(m, T):
    """
    Compute the Erlang B blocking probability for m servers and offered traffic T.
    """
    from math import factorial

    numerator = (T ** m) / factorial(m)
    denominator = sum((T ** i) / factorial(i) for i in range(m + 1))
    return numerator / denominator

# Example usage:
m = 10
T = 8  # offered traffic in Erlangs
blocking_probability = erlang_b(m, T)
print(f"Erlang B blocking probability (m={m}, T={T}): {blocking_probability:.4f}")

Erlang B blocking probability (m=10, T=8): 0.1217


question 2

In [7]:


class BlockingSystemSimulation_2:
    def __init__(self, m, mean_service_time, mean_interarrival_time, num_customers):
        self.m = m  # number of service units
        self.mean_service_time = mean_service_time
        self.mean_interarrival_time = mean_interarrival_time
        self.num_customers = num_customers
        self.blocked_customers = 0
        self.service_times = []
        self.arrival_times = []
        self.service_start_times = []
        self.service_end_times = []
    


    def simulate(self, erlang_k=2):
        lmbda = erlang_k
        arrival_time = 0

        for _ in range(self.num_customers):
            # Always use Erlang (Gamma) interarrival times
            interarrival_time = np.random.gamma(shape=erlang_k, scale=1/lmbda)
            arrival_time += interarrival_time
            self.arrival_times.append(arrival_time)

            # Generate service time
            service_time = np.random.exponential(self.mean_service_time)
            self.service_times.append(service_time)

            # Remove completed services
            self.service_end_times = [end_time for end_time in self.service_end_times if end_time > arrival_time]

            # Check for available service units
            if len(self.service_end_times) < self.m:
                # Service can start immediately
                service_start_time = arrival_time
                service_end_time = service_start_time + service_time
                self.service_start_times.append(service_start_time)
                self.service_end_times.append(service_end_time)
            else:
                # All service units are busy, customer is blocked
                self.blocked_customers += 1
    

    def fraction_blocked(self):
        return self.blocked_customers / self.num_customers

    def confidence_interval(self, confidence=0.95):
        p = self.fraction_blocked()
        n = self.num_customers
        z = stats.norm.ppf((1 + confidence) / 2)
        se = np.sqrt(p * (1 - p) / n)
        return p - z * se, p + z * se

if __name__ == "__main__":
    m = 10
    mean_service_time = 8
    mean_interarrival_time = 1
    num_customers = 10000
    block_list =[]
    k = 8  # or any integer > 1 for Erlang-k
    lmbda = k  # so mean = k / lmbda = 1
    
    for i in range(10):
        simulation = BlockingSystemSimulation_2(m, mean_service_time, mean_interarrival_time, num_customers)
        simulation.simulate(k)
        block_list.append(simulation.fraction_blocked())
        print(f"Simulation {i+1}: Blocked customers = {simulation.fraction_blocked():.4f}")
    

    blocked_fraction = simulation.fraction_blocked()
    ci_lower, ci_upper = simulation.confidence_interval()

    print(f"Fraction of blocked customers: {blocked_fraction:.4f}")
    print(f"95% Confidence interval for blocked fraction: ({ci_lower:.4f}, {ci_upper:.4f})")

Simulation 1: Blocked customers = 0.0693
Simulation 2: Blocked customers = 0.0787
Simulation 3: Blocked customers = 0.0726
Simulation 4: Blocked customers = 0.0684
Simulation 5: Blocked customers = 0.0622
Simulation 6: Blocked customers = 0.0626
Simulation 7: Blocked customers = 0.0690
Simulation 8: Blocked customers = 0.0811
Simulation 9: Blocked customers = 0.0688
Simulation 10: Blocked customers = 0.0684
Fraction of blocked customers: 0.0684
95% Confidence interval for blocked fraction: (0.0635, 0.0733)


In [8]:
if __name__ == "__main__":
    m = 10
    mean_service_time = 8
    mean_interarrival_time = 1
    num_customers = 10000
    for k in range(1, 11):  # k from 1 to 10
        lmbda = k
        block_list = []
        for i in range(10):
            simulation = BlockingSystemSimulation_2(m, mean_service_time, mean_interarrival_time, num_customers)
            simulation.simulate(erlang_k=k)
            block_list.append(simulation.fraction_blocked())
        ci_lower, ci_upper = confidence_interval_t(block_list)
        sum_blocked = sum(block_list)
        mean_blocked = np.mean(block_list)
        print(f"k={k}: Mean blocked fraction = {mean_blocked:.4f}, 95% CI = ({ci_lower:.4f}, {ci_upper:.4f})")

k=1: Mean blocked fraction = 0.1187, 95% CI = (0.1155, 0.1218)
k=2: Mean blocked fraction = 0.0896, 95% CI = (0.0865, 0.0927)
k=3: Mean blocked fraction = 0.0833, 95% CI = (0.0803, 0.0864)
k=4: Mean blocked fraction = 0.0788, 95% CI = (0.0762, 0.0813)
k=5: Mean blocked fraction = 0.0725, 95% CI = (0.0701, 0.0749)
k=6: Mean blocked fraction = 0.0723, 95% CI = (0.0696, 0.0750)
k=7: Mean blocked fraction = 0.0709, 95% CI = (0.0682, 0.0735)
k=8: Mean blocked fraction = 0.0682, 95% CI = (0.0655, 0.0709)
k=9: Mean blocked fraction = 0.0681, 95% CI = (0.0648, 0.0715)
k=10: Mean blocked fraction = 0.0640, 95% CI = (0.0613, 0.0666)


question 2b

In [9]:
class BlockingSystemSimulation_2b:
    def __init__(self, m, mean_service_time, mean_interarrival_time, num_customers):
        self.m = m  # number of service units
        self.mean_service_time = mean_service_time
        self.mean_interarrival_time = mean_interarrival_time
        self.num_customers = num_customers
        self.blocked_customers = 0
        self.service_times = []
        self.arrival_times = []
        self.service_start_times = []
        self.service_end_times = []
    


    def simulate(self):
        arrival_time = 0
        p1, lmbda1 = 0.8, 0.8333
        p2, lmbda2 = 0.2, 5.0
        for _ in range(self.num_customers):
            # Generate hyper-exponential interarrival time
            if np.random.rand() < p1:
                interarrival_time = np.random.exponential(1 / lmbda1)
            else:
                interarrival_time = np.random.exponential(1 / lmbda2)
            arrival_time += interarrival_time
            self.arrival_times.append(arrival_time)

            # Remove completed services
            self.service_end_times = [end_time for end_time in self.service_end_times if end_time > arrival_time]

            # Check for available service units
            if len(self.service_end_times) < self.m:
                service_time = np.random.exponential(self.mean_service_time)
                self.service_times.append(service_time)
                service_start_time = arrival_time
                service_end_time = service_start_time + service_time
                self.service_start_times.append(service_start_time)
                self.service_end_times.append(service_end_time)
            else:
                self.blocked_customers += 1
    

    def fraction_blocked(self):
        return self.blocked_customers / self.num_customers

    def confidence_interval(self, confidence=0.95):
        p = self.fraction_blocked()
        n = self.num_customers
        z = stats.norm.ppf((1 + confidence) / 2)
        se = np.sqrt(p * (1 - p) / n)
        return p - z * se, p + z * se

In [10]:
if __name__ == "__main__":
    m = 10
    mean_service_time = 8
    mean_interarrival_time = 1
    num_customers = 10000
    block_list = []

    for i in range(10):
        simulation = BlockingSystemSimulation_2b(m, mean_service_time, mean_interarrival_time, num_customers)
        simulation.simulate()  # No arguments needed for hyper-exponential
        block_list.append(simulation.fraction_blocked())
        print(f"Simulation {i+1}: Blocked customers = {simulation.fraction_blocked():.4f}")

    blocked_fraction = np.mean(block_list)
    # Use t-confidence interval for the mean of block_list
    ci_lower, ci_upper = confidence_interval_t(block_list)

    print(f"Mean fraction of blocked customers: {blocked_fraction:.4f}")
    print(f"95% Confidence interval for mean blocked fraction: ({ci_lower:.4f}, {ci_upper:.4f})")

Simulation 1: Blocked customers = 0.1356
Simulation 2: Blocked customers = 0.1362
Simulation 3: Blocked customers = 0.1337
Simulation 4: Blocked customers = 0.1438
Simulation 5: Blocked customers = 0.1453
Simulation 6: Blocked customers = 0.1335
Simulation 7: Blocked customers = 0.1345
Simulation 8: Blocked customers = 0.1313
Simulation 9: Blocked customers = 0.1340
Simulation 10: Blocked customers = 0.1395
Mean fraction of blocked customers: 0.1367
95% Confidence interval for mean blocked fraction: (0.1334, 0.1401)


Question 3a

In [11]:
class BlockingSystemSimulation_3a:
    def __init__(self, m, mean_service_time, mean_interarrival_time, num_customers, sigma=1):
        self.m = m  # number of service units
        self.mean_service_time = mean_service_time
        self.mean_interarrival_time = mean_interarrival_time
        self.num_customers = num_customers
        self.blocked_customers = 0
        self.service_times = []
        self.arrival_times = []
        self.service_start_times = []
        self.service_end_times = []
        self.mu = np.log(mean_service_time)
        self.sigma = sigma

    def simulate(self):
        arrival_time = 0
        for _ in range(self.num_customers):
            # Poisson arrivals (exponential interarrival times)
            interarrival_time = np.random.exponential(self.mean_interarrival_time)
            arrival_time += interarrival_time
            self.arrival_times.append(arrival_time)

            # Remove completed services before checking for available units
            self.service_end_times = [end_time for end_time in self.service_end_times if end_time > arrival_time]

            # Check for available service units
            if len(self.service_end_times) < self.m:
                # Constant service time
                service_time = np.random.lognormal(self.mu, self.sigma)
                self.service_times.append(service_time)
                service_start_time = arrival_time
                service_end_time = service_start_time + service_time
                self.service_start_times.append(service_start_time)
                self.service_end_times.append(service_end_time)
            else:
                self.blocked_customers += 1

    def fraction_blocked(self):
        return self.blocked_customers / self.num_customers

    def confidence_interval(self, confidence=0.95):
        p = self.fraction_blocked()
        n = self.num_customers
        z = stats.norm.ppf((1 + confidence) / 2)
        se = np.sqrt(p * (1 - p) / n)
        return p - z * se, p + z * se

In [12]:
if __name__ == "__main__":
    m = 10
    mean_service_time = 8
    mean_interarrival_time = 1
    num_customers = 10000
    block_list = []

    for i in range(10):
        simulation = BlockingSystemSimulation_3a(m, mean_service_time, mean_interarrival_time, num_customers)
        simulation.simulate()
        block_list.append(simulation.fraction_blocked())
        print(f"Simulation {i+1}: Blocked customers = {simulation.fraction_blocked():.4f}")

    blocked_fraction = np.mean(block_list)
    ci_lower, ci_upper = confidence_interval_t(block_list)
    print(f"Mean fraction of blocked customers: {blocked_fraction:.4f}")
    print(f"95% Confidence interval for mean blocked fraction: ({ci_lower:.4f}, {ci_upper:.4f})")

Simulation 1: Blocked customers = 0.3476
Simulation 2: Blocked customers = 0.3475
Simulation 3: Blocked customers = 0.3482
Simulation 4: Blocked customers = 0.3496
Simulation 5: Blocked customers = 0.3471
Simulation 6: Blocked customers = 0.3532
Simulation 7: Blocked customers = 0.3413
Simulation 8: Blocked customers = 0.3422
Simulation 9: Blocked customers = 0.3559
Simulation 10: Blocked customers = 0.3621
Mean fraction of blocked customers: 0.3495
95% Confidence interval for mean blocked fraction: (0.3450, 0.3539)


3b

In [13]:
class BlockingSystemSimulation_Pareto:
    def __init__(self, m, mean_service_time, mean_interarrival_time, num_customers, pareto_k):
        self.m = m
        self.mean_service_time = mean_service_time
        self.mean_interarrival_time = mean_interarrival_time
        self.num_customers = num_customers
        self.pareto_k = pareto_k
        self.blocked_customers = 0
        self.service_times = []
        self.arrival_times = []
        self.service_start_times = []
        self.service_end_times = []

        # Calculate scale xm so that mean matches mean_service_time
        self.pareto_xm = mean_service_time * (pareto_k - 1) / pareto_k

    def simulate(self):
        arrival_time = 0
        for _ in range(self.num_customers):
            interarrival_time = np.random.exponential(self.mean_interarrival_time)
            arrival_time += interarrival_time
            self.arrival_times.append(arrival_time)

            # Remove completed services before checking for available units
            self.service_end_times = [end_time for end_time in self.service_end_times if end_time > arrival_time]

            if len(self.service_end_times) < self.m:
                # Pareto service time
                service_time = np.random.pareto(self.pareto_k) * self.pareto_xm + self.pareto_xm
                self.service_times.append(service_time)
                service_start_time = arrival_time
                service_end_time = service_start_time + service_time
                self.service_start_times.append(service_start_time)
                self.service_end_times.append(service_end_time)
            else:
                self.blocked_customers += 1

    def fraction_blocked(self):
        return self.blocked_customers / self.num_customers

    def confidence_interval(self, confidence=0.95):
        p = self.fraction_blocked()
        n = self.num_customers
        z = stats.norm.ppf((1 + confidence) / 2)
        se = np.sqrt(p * (1 - p) / n)
        return p - z * se, p + z * se


In [20]:

# Example usage for k = 1.05 and k = 2.05
for pareto_k in [1.05, 2.05]:
    print(f"\nPareto service times with k = {pareto_k}")
    m = 10
    mean_service_time = 8
    mean_interarrival_time = 1
    num_customers = 10000
    block_list = []

    for i in range(10):
        simulation = BlockingSystemSimulation_Pareto(m, mean_service_time, mean_interarrival_time, num_customers, pareto_k)
        simulation.simulate()
        block_list.append(simulation.fraction_blocked())
        print(f"Simulation {i+1}: Blocked customers = {simulation.fraction_blocked():.4f}")

    blocked_fraction = np.mean(block_list)
    ci_lower, ci_upper = confidence_interval_t(block_list)
    print(f"Mean fraction of blocked customers: {blocked_fraction:.4f}")
    print(f"95% Confidence interval for mean blocked fraction: ({ci_lower:.4f}, {ci_upper:.4f})")


Pareto service times with k = 1.05
Simulation 1: Blocked customers = 0.0009
Simulation 2: Blocked customers = 0.0007
Simulation 3: Blocked customers = 0.0008
Simulation 4: Blocked customers = 0.0000
Simulation 5: Blocked customers = 0.0035
Simulation 6: Blocked customers = 0.0015
Simulation 7: Blocked customers = 0.0004
Simulation 8: Blocked customers = 0.0006
Simulation 9: Blocked customers = 0.0009
Simulation 10: Blocked customers = 0.0003
Mean fraction of blocked customers: 0.0010
95% Confidence interval for mean blocked fraction: (0.0003, 0.0017)

Pareto service times with k = 2.05
Simulation 1: Blocked customers = 0.1188
Simulation 2: Blocked customers = 0.1111
Simulation 3: Blocked customers = 0.1217
Simulation 4: Blocked customers = 0.1207
Simulation 5: Blocked customers = 0.1272
Simulation 6: Blocked customers = 0.1166
Simulation 7: Blocked customers = 0.1156
Simulation 8: Blocked customers = 0.1111
Simulation 9: Blocked customers = 0.1131
Simulation 10: Blocked customers = 0.

3c


In [23]:
class BlockingSystemSimulation_erlang:
    def __init__(self, m, mean_service_time, mean_interarrival_time, num_customers, sigma=1.0):
        self.m = m
        self.mean_service_time = mean_service_time
        self.mean_interarrival_time = mean_interarrival_time
        self.num_customers = num_customers
        self.sigma = sigma  # shape parameter for log-normal
        self.blocked_customers = 0
        self.service_times = []
        self.arrival_times = []
        self.service_start_times = []
        self.service_end_times = []

        # Calculate mu so that mean matches mean_service_time
        # mean = exp(mu + sigma^2 / 2) => mu = log(mean) - sigma^2 / 2
        self.mu = np.log(mean_service_time) - (sigma**2) / 2

    def simulate(self, erlang_k=8, lmbda=1):
        arrival_time = 0
        for _ in range(self.num_customers):
            interarrival_time = np.random.exponential(self.mean_interarrival_time)
            arrival_time += interarrival_time
            self.arrival_times.append(arrival_time)

            # Remove completed services before checking for available units
            self.service_end_times = [end_time for end_time in self.service_end_times if end_time > arrival_time]

            if len(self.service_end_times) < self.m:
                # Log-normal service time
                service_time = np.random.gamma(shape=erlang_k, scale=1/lmbda)
                self.service_times.append(service_time)
                service_start_time = arrival_time
                service_end_time = service_start_time + service_time
                self.service_start_times.append(service_start_time)
                self.service_end_times.append(service_end_time)
            else:
                self.blocked_customers += 1

    def fraction_blocked(self):
        return self.blocked_customers / self.num_customers

    def confidence_interval(self, confidence=0.95):
        p = self.fraction_blocked()
        n = self.num_customers
        z = stats.norm.ppf((1 + confidence) / 2)
        se = np.sqrt(p * (1 - p) / n)
        return p - z * se, p + z * se


In [24]:

# Example usage:
if __name__ == "__main__":
    m = 10
    mean_service_time = 8
    mean_interarrival_time = 1
    num_customers = 10000
    sigma = 1.0  # You can try other values for more/less variability
    block_list = []

    for i in range(10):
        simulation = BlockingSystemSimulation_erlang(m, mean_service_time, mean_interarrival_time, num_customers, sigma)
        simulation.simulate()
        block_list.append(simulation.fraction_blocked())
        print(f"Simulation {i+1}: Blocked customers = {simulation.fraction_blocked():.4f}")

    blocked_fraction = np.mean(block_list)
    ci_lower, ci_upper = confidence_interval_t(block_list)
    print(f"Mean fraction of blocked customers: {blocked_fraction:.4f}")
    print(f"95% Confidence interval for mean blocked fraction: ({ci_lower:.4f}, {ci_upper:.4f})")

Simulation 1: Blocked customers = 0.1196
Simulation 2: Blocked customers = 0.1247
Simulation 3: Blocked customers = 0.1223
Simulation 4: Blocked customers = 0.1207
Simulation 5: Blocked customers = 0.1261
Simulation 6: Blocked customers = 0.1280
Simulation 7: Blocked customers = 0.1229
Simulation 8: Blocked customers = 0.1274
Simulation 9: Blocked customers = 0.1300
Simulation 10: Blocked customers = 0.1182
Mean fraction of blocked customers: 0.1240
95% Confidence interval for mean blocked fraction: (0.1212, 0.1268)


In [27]:
class BlockingSystemSimulation_uniform:
    def __init__(self, m, mean_service_time, mean_interarrival_time, num_customers, a=4, b=12):
        self.m = m
        self.mean_service_time = mean_service_time
        self.mean_interarrival_time = mean_interarrival_time
        self.num_customers = num_customers
        self.blocked_customers = 0
        self.service_times = []
        self.arrival_times = []
        self.service_start_times = []
        self.service_end_times = []
        self.a = a
        self.b = b

        # Calculate mu so that mean matches mean_service_time
        # mean = exp(mu + sigma^2 / 2) => mu = log(mean) - sigma^2 / 2
        self.mu = np.log(mean_service_time) - (sigma**2) / 2

    def simulate(self):
        arrival_time = 0
        for _ in range(self.num_customers):
            interarrival_time = np.random.exponential(self.mean_interarrival_time)
            arrival_time += interarrival_time
            self.arrival_times.append(arrival_time)

            # Remove completed services before checking for available units
            self.service_end_times = [end_time for end_time in self.service_end_times if end_time > arrival_time]

            if len(self.service_end_times) < self.m:
                # Log-normal service time
                service_time = np.random.uniform(self.a, self.b)
                self.service_times.append(service_time)
                service_start_time = arrival_time
                service_end_time = service_start_time + service_time
                self.service_start_times.append(service_start_time)
                self.service_end_times.append(service_end_time)
            else:
                self.blocked_customers += 1

    def fraction_blocked(self):
        return self.blocked_customers / self.num_customers

    def confidence_interval(self, confidence=0.95):
        p = self.fraction_blocked()
        n = self.num_customers
        z = stats.norm.ppf((1 + confidence) / 2)
        se = np.sqrt(p * (1 - p) / n)
        return p - z * se, p + z * se


In [28]:

# Example usage:
if __name__ == "__main__":
    m = 10
    mean_service_time = 8
    mean_interarrival_time = 1
    num_customers = 10000
    sigma = 1.0  # You can try other values for more/less variability
    block_list = []

    for i in range(10):
        simulation = BlockingSystemSimulation_uniform(m, mean_service_time, mean_interarrival_time, num_customers)
        simulation.simulate()
        block_list.append(simulation.fraction_blocked())
        print(f"Simulation {i+1}: Blocked customers = {simulation.fraction_blocked():.4f}")

    blocked_fraction = np.mean(block_list)
    ci_lower, ci_upper = confidence_interval_t(block_list)
    print(f"Mean fraction of blocked customers: {blocked_fraction:.4f}")
    print(f"95% Confidence interval for mean blocked fraction: ({ci_lower:.4f}, {ci_upper:.4f})")

Simulation 1: Blocked customers = 0.1204
Simulation 2: Blocked customers = 0.1194
Simulation 3: Blocked customers = 0.1226
Simulation 4: Blocked customers = 0.1230
Simulation 5: Blocked customers = 0.1175
Simulation 6: Blocked customers = 0.1366
Simulation 7: Blocked customers = 0.1226
Simulation 8: Blocked customers = 0.1168
Simulation 9: Blocked customers = 0.1286
Simulation 10: Blocked customers = 0.1201
Mean fraction of blocked customers: 0.1228
95% Confidence interval for mean blocked fraction: (0.1185, 0.1270)
